In [ ]:
!pip install tensorflow numpy matplotlib opencv-python
!pip install kaggle

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os


In [6]:
from google.colab import files
files.upload()


ModuleNotFoundError: No module named 'google.colab'

In [7]:
import os
import shutil


os.makedirs("/root/.kaggle", exist_ok=True)


shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")


os.chmod("/root/.kaggle/kaggle.json", 600)


In [8]:
!kaggle datasets download abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0




  0%|          | 0.00/2.04G [00:00<?, ?B/s]
  0%|          | 1.00M/2.04G [00:01<46:31, 784kB/s]
  0%|          | 2.00M/2.04G [00:01<23:04, 1.58MB/s]
  0%|          | 3.00M/2.04G [00:01<14:22, 2.54MB/s]
  0%|          | 4.00M/2.04G [00:01<10:32, 3.46MB/s]
  0%|          | 6.00M/2.04G [00:01<06:22, 5.70MB/s]
  0%|          | 8.00M/2.04G [00:02<04:56, 7.36MB/s]
  0%|          | 10.0M/2.04G [00:02<04:13, 8.60MB/s]
  1%|          | 12.0M/2.04G [00:02<03:47, 9.55MB/s]
  1%|          | 14.0M/2.04G [00:02<03:34, 10.2MB/s]
  1%|          | 16.0M/2.04G [00:02<03:24, 10.6MB/s]
  1%|          | 18.0M/2.04G [00:02<03:16, 11.0MB/s]
  1%|          | 20.0M/2.04G [00:03<03:12, 11.2MB/s]
  1%|          | 22.0M/2.04G [00:03<03:16, 11.0MB/s]
  1%|          | 24.0M/2.04G [00:03<03:10, 11.4MB/s]
  1%|          | 26.0M/2.04G [00:03<03:39, 9.85MB/s]
  1%|▏         | 27.0M/2.04G [00:03<03:38, 9.89MB/s]
  1%|▏         | 29.0M/2.04G [00:04<03:25, 10.5MB/s]
  1%|▏         | 31.0M/2.04G [00:04<03:17, 10.9MB/s]
  

In [9]:
import zipfile

with zipfile.ZipFile("plantvillage-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("plantvillage")


In [10]:
import os

# Set the dataset directory path
dataset_path = "plantvillage"

# Get the list of class names (folder names)
class_names = sorted(os.listdir(dataset_path))
print("Class Names:", class_names)


Class Names: ['plantvillage dataset']


In [ ]:
import os

# List contents
print(os.listdir("plantvillage"))



In [ ]:
!kaggle datasets download mohitsingh1804/plantvillage

In [ ]:
import zipfile

with zipfile.ZipFile("plantvillage.zip", 'r') as zip_ref:
    zip_ref.extractall("plantvillage_data")


In [ ]:
import os


print(os.listdir("plantvillage_data"))


In [ ]:
from tensorflow import keras


IMG_SIZE = (224, 224)
BATCH_SIZE = 32


datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_data = datagen.flow_from_directory(
    "/content/plantvillage_data/PlantVillage/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)


val_data = datagen.flow_from_directory(
    "/content/plantvillage_data/PlantVillage/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)


In [ ]:
print("Classes:", train_data.class_indices)


In [ ]:
import matplotlib.pyplot as plt

images, labels = next(train_data)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i])
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

print("CNN model built.")

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Model compiled.")

model.summary()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf


IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "/content/plantvillage_data/PlantVillage/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_datagen.flow_from_directory(
    "/content/plantvillage_data/PlantVillage/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]


history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=callbacks
)


base_model.trainable = True
model.compile(optimizer=optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)


loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf


IMG_SIZE = (160, 160)
BATCH_SIZE = 16
EPOCHS = 10


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)


train_data = train_datagen.flow_from_directory(
    "/content/plantvillage_data/PlantVillage/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_datagen.flow_from_directory(
    "/content/plantvillage_data/PlantVillage/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(160, 160, 3))
base_model.trainable = False


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]


history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=callbacks
)


base_model.trainable = True
model.compile(optimizer=optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)


loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
